<a href="https://colab.research.google.com/github/HighCWu/anime_biggan_toy/blob/main/colab/AnimeBigGAN_tf2pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Model

In [ ]:
!rsync --verbose --progress rsync://176.9.41.242:873/biggan/2020-05-18-spresser-biggan-256px-danbooruplus-run39-607250.tar.xz ./

In [ ]:
!tar -xf 2020-05-18-spresser-biggan-256px-danbooruplus-run39-607250.tar.xz

## Restore Model

In [ ]:
!pip install mock
!git clone https://github.com/HighCWu/compare_gan
!cd compare_gan && git checkout easyhub
!cd compare_gan && pip install -e .
!cp ./2020-05-18-spresser-biggan-256px-danbooruplus-run39-607250 ./compare_gan/ -r

In [ ]:
import os
os.chdir('compare_gan')

In [ ]:
%tensorflow_version 1.x

In [ ]:
# ModularGAN extend build_graph fn
import tensorflow as tf
from compare_gan.gans.modular_gan import ModularGAN


def build_graph(self, model=None):
    batch_size = None
    is_training = False
    y_gen, y_disc = None, None
    inputs_gen, inputs_disc = {}, {}
    outputs_gen, outputs_disc = {}, {}
    inputs_gen["z"] = None if model=='disc' else tf.placeholder(
        shape=(batch_size, self._z_dim),
        dtype=tf.float32,
        name="z_for_eval"
    )
    inputs_disc["images"] = None if model=='gen' else tf.placeholder(
        shape=[batch_size] + list(self._dataset.image_shape),
        dtype=tf.float32,
        name="images_for_eval"
    )
    if self.conditional:
        inputs_gen["labels"] = None if model=='disc' else tf.placeholder(
            shape=(batch_size,),
            dtype=tf.int32,
            name="labels_for_gen_eval"
        )
        y_gen = None if model=='disc' else self._get_one_hot_labels(inputs_gen["labels"])
        inputs_disc["labels"] = None if model=='gen' else tf.placeholder(
            shape=(batch_size,),
            dtype=tf.int32,
            name="labels_for_disc_eval"
        )
        y_disc = None if model=='gen' else self._get_one_hot_labels(inputs_disc["labels"])
    else:
      y_gen, y_disc = None, None

    outputs_disc["prediction"], _, _ = (None,None,None) if model=='gen' else self.discriminator(
        inputs_disc["images"], y=y_disc, is_training=is_training
    )

    z = inputs_gen["z"]
    generated = None if model=='disc' else self.generator(z=z, y=y_gen, is_training=is_training)
    if self._g_use_ema and model != 'disc':
        g_vars = [var for var in tf.trainable_variables()
              if "generator" in var.name]
        ema = tf.train.ExponentialMovingAverage(decay=self._ema_decay)
        # Create the variables that will be loaded from the checkpoint.
        ema.apply(g_vars)
        def ema_getter(getter, name, *args, **kwargs):
            var = getter(name, *args, **kwargs)
            ema_var = ema.average(var)
            if ema_var is None:
                var_names_without_ema = {"u_var", "accu_mean", "accu_variance",
                              "accu_counter", "update_accus"}
                if name.split("/")[-1] not in var_names_without_ema:
                    print("Could not find EMA variable for %s.", name)
                return var
            return ema_var
        with tf.variable_scope("", values=[z, y_gen], reuse=True, custom_getter=ema_getter):
            ema_generated = self.generator(z, y=y_gen, is_training=is_training)
            if not is_training:
                generated = ema_generated
    outputs_gen["generated"] = None if model=='disc' else generated
    return {
        "gen": {
            "inputs": inputs_gen, 
            "outputs": outputs_gen
        },
        "disc": {
            "inputs": inputs_disc,
            "outputs": outputs_disc
        }
    }
ModularGAN.build_graph = build_graph

In [ ]:
import os
import gin
import tensorflow as tf
import tensorflow_hub as hub
from compare_gan import main, runner_lib, datasets
from compare_gan.gans import utils, modular_gan
from compare_gan.gans.modular_gan import ModularGAN

os.environ["NUM_CORES"] = '8'
gin.parse_config_file('example_configs/bigrun40.gin')
gin.bind_parameter("standardize_batch.use_cross_replica_mean", False)
gin.bind_parameter("spectral_norm.save_in_checkpoint", False)
src_model_path = '2020-05-18-spresser-biggan-256px-danbooruplus-run39-607250/'

use_ema = True

class FakeDataset:
    num_classes = 1000
    random_labels = True
    image_shape = (256, 256, 3)


model = ModularGAN(FakeDataset, {
        "architecture": "resnet_biggan_arch",
        "z_dim": 140,
        "lambda": 1,
    }, src_model_path, g_use_ema=use_ema
)

In [ ]:
import os
import itertools
import numpy as np
from PIL import Image
from compare_gan.architectures import arch_ops
arch_ops.use_assign_forbidden = False

os.makedirs('accu_samples', exist_ok=True)

batch_size = 8
num_accu_steps = 8 # a few steps and accu all levels together
checkpoint_path = os.path.join('anime-biggan-256px-run39-607250-tmp', "model.ckpt")
tf.keras.backend.clear_session()


sess = tf.Session()
inout_nodes = model.build_graph()
inout_gen, inout_disc = inout_nodes["gen"], inout_nodes["disc"]
init_op = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(init_op)
saver = tf.train.Saver()
saver.restore(sess, src_model_path + 'model.ckpt-607250')
update_accu_switches = [v for v in tf.global_variables()
                if "accu/update_accus" in v.name]
def run_model():
    z = rng.randn(batch_size, model._z_dim).astype(np.float32)
    if not model.conditional:
        _z = inout_gen['inputs']['z']
        images = sess.run(inout_gen['outputs']['generated'], feed_dict={_z: z})
    else:
        y = rng.randint(model._dataset.num_classes, size=(batch_size, )).astype(np.int32)
        _z = inout_gen['inputs']['z']
        _y_gen = inout_gen['inputs']['labels']
        images = sess.run(inout_gen['outputs']['generated'], feed_dict={_z: z, _y_gen: y})
    return images
for j, update_accu in enumerate(update_accu_switches):
    sess.run(tf.assign(update_accu, 1))

rng = np.random.RandomState(0)
for i in range(num_accu_steps):
    images = run_model()
    dst_filename = f'accu_samples/batchsize_{batch_size}.png'
    image = Image.fromarray(np.uint8(images.transpose((1,0,2,3)).reshape((256,-1,3)).clip(0,1) * 255))
    image.save(f'accu_samples/batchsize_{batch_size}_accu_{str(i+1).zfill(5)}.png')
    print(f'Updating BN accumulators {str(i+1).zfill(5)}/{str(num_accu_steps).zfill(5)} steps.')

for j, update_accu in enumerate(update_accu_switches):
    sess.run(tf.assign(update_accu, 0))

rng = np.random.RandomState(0)
images = run_model()
dst_filename = f'accu_samples/batchsize_{batch_size}.png'
image = Image.fromarray(np.uint8(images.transpose((1,0,2,3)).reshape((256,-1,3)).clip(0,1) * 255))
image.save(dst_filename)
print('Done updating BN accumulators.')


for tensor in [tensor for op in sess.graph.get_operations() for tensor in op.values()]:
    if 'generator_1' in tensor.name: # ema
        print(tensor.name, tensor.shape)

In [ ]:
for tensor in [tensor for op in sess.graph.get_operations() for tensor in op.values()]:
  if 'generator' in tensor.name:
    print(tensor.name, tensor.shape)

## TFHub Model to PyTorch Model

In [ ]:
var_list = []
for var in tf.global_variables():
    if 'discriminator' not in var.name:
        val = sess.run(var)
        if '/ExponentialMovingAverage' in var.name:
            name = ''.join(var.name.split('/ExponentialMovingAverage'))
            for i, weights in enumerate(var_list):
                if weights[0] == name:
                    weights[1] = val
        else:
            var_list.append([var.name, val])

idx = 0
var_list_ids = []
for var in tf.local_variables():
    if 'discriminator' not in var.name and \
      '/u_var' in var.name and \
      'bias' not in var.name and \
      'accu_' not in var.name and \
      'embed_y' not in var.name and \
      'final_norm' not in var.name:
        while var.name.split('/u_var')[0] + ':0' != var_list[idx][0] and var.name.split('/u_var')[0] != var_list[idx][0]:
            idx = idx + 1
        var_list_ids.append([idx, [var.name, sess.run(var)]])
for (idx, var) in var_list_ids[::-1]:
    var_list.insert(idx + 1, var)

for weights in var_list:
  print(weights[0], weights[1].shape)

tf_weights = var_list
 
def tf_filter(x):
  if 'accu/update_accus:0' in x[0]:
    return False
  return True
 
tf_weights = list(filter(tf_filter, tf_weights))

In [ ]:
#@title Define Generator and Discriminator model
import numpy as np
import torch
from torch import nn
from torch.nn import Parameter
from torch.nn import functional as F


def l2_normalize(v, dim=None, eps=1e-12):
    return v / (v.norm(dim=dim, keepdim=True) + eps)
    
 
def unpool(value):
    """Unpooling operation.
    N-dimensional version of the unpooling operation from
    https://www.robots.ox.ac.uk/~vgg/rg/papers/Dosovitskiy_Learning_to_Generate_2015_CVPR_paper.pdf
    Taken from: https://github.com/tensorflow/tensorflow/issues/2169
    Args:
        value: a Tensor of shape [b, d0, d1, ..., dn, ch]
        name: name of the op
    Returns:
        A Tensor of shape [b, 2*d0, 2*d1, ..., 2*dn, ch]
    """
    value = torch.Tensor.permute(value, [0,2,3,1])
    sh = list(value.shape)
    dim = len(sh[1:-1])
    out = (torch.reshape(value, [-1] + sh[-dim:]))
    for i in range(dim, 0, -1):
        out = torch.cat([out, torch.zeros_like(out)], i)
    out_size = [-1] + [s * 2 for s in sh[1:-1]] + [sh[-1]]
    out = torch.reshape(out, out_size)
    out = torch.Tensor.permute(out, [0,3,1,2])
    return out
 
 
class BatchNorm2d(nn.BatchNorm2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.initialized = False
        self.accumulating = False
        self.accumulated_mean = Parameter(torch.zeros(args[0]), requires_grad=False)
        self.accumulated_var = Parameter(torch.zeros(args[0]), requires_grad=False)
        self.accumulated_counter = Parameter(torch.zeros(1)+1e-12, requires_grad=False)
 
    def forward(self, inputs, *args, **kwargs):
        if not self.initialized:
            self.check_accumulation()
            self.set_initialized(True)
        if self.accumulating:
            self.eval()
            with torch.no_grad():
                axes = [0] + ([] if len(inputs.shape) == 2 else list(range(2,len(inputs.shape))))
                _mean = torch.mean(inputs, axes, keepdim=True)
                mean = torch.mean(inputs, axes, keepdim=False)
                var = torch.mean((inputs-_mean)**2, axes)
                self.accumulated_mean.copy_(self.accumulated_mean + mean)
                self.accumulated_var.copy_(self.accumulated_var + var)
                self.accumulated_counter.copy_(self.accumulated_counter + 1)
                _mean = self.running_mean*1.0
                _variance = self.running_var*1.0
                self._mean.copy_(self.accumulated_mean / self.accumulated_counter)
                self._variance.copy_(self.accumulated_var / self.accumulated_counter)
                out = super().forward(inputs, *args, **kwargs)
                self.running_mean.copy_(_mean)
                self.running_var.copy_(_variance)
                return out
        out = super().forward(inputs, *args, **kwargs)
        return out
 
    def check_accumulation(self):
        if self.accumulated_counter.detach().cpu().numpy().mean() > 1-1e-12:
            self.running_mean.copy_(self.accumulated_mean / self.accumulated_counter)
            self.running_var.copy_(self.accumulated_var / self.accumulated_counter)
            return True
        return False
 
    def clear_accumulated(self):
        self.accumulated_mean.copy_(self.accumulated_mean*0.0)
        self.accumulated_var.copy_(self.accumulated_var*0.0)
        self.accumulated_counter.copy_(self.accumulated_counter*0.0+1e-2)
 
    def set_accumulating(self, status=True):
        if status:
            self.accumulating = True
        else:
            self.accumulating = False
 
    def set_initialized(self, status=False):
        if not status:
            self.initialized = False
        else:
            self.initialized = True
 

class SpectralNorm(nn.Module):
    def __init__(self, module, name='weight', power_iterations=2):
        super().__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()
 
    def _update_u(self):
        w = self.weight
        u = self.weight_u
 
        if len(w.shape) == 4:
            _w = torch.Tensor.permute(w, [2,3,1,0])
            _w = torch.reshape(_w, [-1, _w.shape[-1]])
        elif isinstance(self.module, nn.Linear) or isinstance(self.module, nn.Embedding):
            _w = torch.Tensor.permute(w, [1,0])
            _w = torch.reshape(_w, [-1, _w.shape[-1]])
        else:
            _w = torch.reshape(w, [-1, w.shape[-1]])
            _w = torch.reshape(_w, [-1, _w.shape[-1]])
        singular_value = "left" if _w.shape[0] <= _w.shape[1] else "right"
        norm_dim = 0 if _w.shape[0] <= _w.shape[1] else 1
        for _ in range(self.power_iterations):
            if singular_value == "left":
                v = l2_normalize(torch.matmul(_w.t(), u), dim=norm_dim)
                u = l2_normalize(torch.matmul(_w, v), dim=norm_dim)
            else:
                v = l2_normalize(torch.matmul(u, _w.t()), dim=norm_dim)
                u = l2_normalize(torch.matmul(v, _w), dim=norm_dim)
 
        if singular_value == "left":
            sigma = torch.matmul(torch.matmul(u.t(), _w), v)
        else:
            sigma = torch.matmul(torch.matmul(v, _w), u.t())
        _w = w / sigma.detach()
        setattr(self.module, self.name, _w)
        self.weight_u.copy_(u.detach())
 
    def _made_params(self):
        try:
            self.weight
            self.weight_u
            return True
        except AttributeError:
            return False
 
    def _make_params(self):
        w = getattr(self.module, self.name)
 
        if len(w.shape) == 4:
            _w = torch.Tensor.permute(w, [2,3,1,0])
            _w = torch.reshape(_w, [-1, _w.shape[-1]])
        elif isinstance(self.module, nn.Linear) or isinstance(self.module, nn.Embedding):
            _w = torch.Tensor.permute(w, [1,0])
            _w = torch.reshape(_w, [-1, _w.shape[-1]])
        else:
            _w = torch.reshape(w, [-1, w.shape[-1]])
        singular_value = "left" if _w.shape[0] <= _w.shape[1] else "right"
        norm_dim = 0 if _w.shape[0] <= _w.shape[1] else 1
        u_shape = (_w.shape[0], 1) if singular_value == "left" else (1, _w.shape[-1])
        
        u = Parameter(w.data.new(*u_shape).normal_(0, 1), requires_grad=False)
        u.copy_(l2_normalize(u, dim=norm_dim).detach())
 
        del self.module._parameters[self.name]
        self.weight = w
        self.weight_u = u
 
    def forward(self, *args, **kwargs):
        self._update_u()
        return self.module.forward(*args, **kwargs)
    
    
class SelfAttention(nn.Module):
    def __init__(self, in_dim, activation=torch.relu):
        super().__init__()
        self.chanel_in = in_dim
        self.activation = activation
    
        self.theta = SpectralNorm(nn.Conv2d(in_dim, in_dim // 8, 1, bias=False))
        self.phi = SpectralNorm(nn.Conv2d(in_dim, in_dim // 8, 1, bias=False))
        self.pool = nn.MaxPool2d(2, 2)
        self.g = SpectralNorm(nn.Conv2d(in_dim, in_dim // 2, 1, bias=False))
        self.o_conv = SpectralNorm(nn.Conv2d(in_dim // 2, in_dim, 1, bias=False))
        self.gamma = Parameter(torch.zeros(1))
    
    def forward(self, x):
        m_batchsize, C, width, height = x.shape
        N = height * width
    
        theta = self.theta(x)
        phi = self.phi(x)
        phi = self.pool(phi)
        phi = torch.reshape(phi,(m_batchsize, -1, N // 4))
        theta = torch.reshape(theta,(m_batchsize, -1, N))
        theta = torch.Tensor.permute(theta,(0, 2, 1))
        attention = torch.softmax(torch.bmm(theta, phi), -1)
        g = self.g(x)
        g = torch.reshape(self.pool(g),(m_batchsize, -1, N // 4))
        attn_g = torch.reshape(torch.bmm(g, torch.Tensor.permute(attention,(0, 2, 1))),(m_batchsize, -1, width, height))
        out = self.o_conv(attn_g)
        return self.gamma * out + x
 
 
class ConditionalBatchNorm2d(nn.Module):
    def __init__(self, num_features, num_classes, eps=1e-5, momentum=0.1):
        super().__init__()
        self.bn_in_cond = BatchNorm2d(num_features, affine=False, eps=eps, momentum=momentum)
        self.gamma_embed = SpectralNorm(nn.Linear(num_classes, num_features, bias=False))
        self.beta_embed = SpectralNorm(nn.Linear(num_classes, num_features, bias=False))
    
    def forward(self, x, y):
        out = self.bn_in_cond(x)

        if isinstance(y, list):
            gamma, beta = y
            out = torch.reshape(gamma, (gamma.shape[0], -1, 1, 1)) * out + torch.reshape(beta, (beta.shape[0], -1, 1, 1))
            return out

        gamma = self.gamma_embed(y)
        # gamma = gamma + 1
        beta = self.beta_embed(y)
        out = torch.reshape(gamma, (gamma.shape[0], -1, 1, 1)) * out + torch.reshape(beta, (beta.shape[0], -1, 1, 1))
        return out
 

class ResBlock(nn.Module):
    def __init__(
        self,
        in_channel,
        out_channel,
        kernel_size=[3, 3],
        padding=1,
        stride=1,
        n_class=None,
        conditional=True,
        activation=torch.relu,
        upsample=True,
        downsample=False,
        z_dim=128,
        use_attention=False,
        skip_proj=None
    ):
        super().__init__()
    
        if conditional:
            self.cond_norm1 = ConditionalBatchNorm2d(in_channel, z_dim)
    
        self.conv0 = SpectralNorm(
            nn.Conv2d(in_channel, out_channel, kernel_size, stride, padding)
        )
    
        if conditional:
            self.cond_norm2 = ConditionalBatchNorm2d(out_channel, z_dim)
    
        self.conv1 = SpectralNorm(
            nn.Conv2d(out_channel, out_channel, kernel_size, stride, padding)
        )
    
        self.skip_proj = False
        if skip_proj is not True and (upsample or downsample):
            self.conv_sc = SpectralNorm(nn.Conv2d(in_channel, out_channel, 1, 1, 0))
            self.skip_proj = True
    
        if use_attention:
            self.attention = SelfAttention(out_channel)
    
        self.upsample = upsample
        self.downsample = downsample
        self.activation = activation
        self.conditional = conditional
        self.use_attention = use_attention
    
    def forward(self, input, condition=None):
        out = input
    
        if self.conditional:
            out = self.cond_norm1(out, condition if not isinstance(condition, list) else condition[0])
        out = self.activation(out)
        if self.upsample:
            out = unpool(out) # out = F.interpolate(out, scale_factor=2)
        out = self.conv0(out)
        if self.conditional:
            out = self.cond_norm2(out, condition if not isinstance(condition, list) else condition[1])
        out = self.activation(out)
        out = self.conv1(out)
        
        if self.downsample:
            out = F.avg_pool2d(out, 2, 2)
    
        if self.skip_proj:
            skip = input
            if self.upsample:
                skip = unpool(skip) # skip = F.interpolate(skip, scale_factor=2)
            skip = self.conv_sc(skip)
            if self.downsample:
                skip = F.avg_pool2d(skip, 2, 2)
            out = out + skip
        else:
            skip = input
    
        if self.use_attention:
            out = self.attention(out)
    
        return out
 
 
class Generator(nn.Module):
    def __init__(self, code_dim=128, n_class=1000, chn=96, blocks_with_attention="B4", resolution=512):
        super().__init__()
    
        def GBlock(in_channel, out_channel, n_class, z_dim, use_attention):
            return ResBlock(in_channel, out_channel, n_class=n_class, z_dim=z_dim, use_attention=use_attention)
    
        self.embed_y = nn.Linear(n_class, 128, bias=False)
    
        self.chn = chn
        self.resolution = resolution 
        self.blocks_with_attention = set(blocks_with_attention.split(",")) 
        self.blocks_with_attention.discard('')
    
        gblock = []
        in_channels, out_channels = self.get_in_out_channels()
        self.num_split = len(in_channels) + 1
    
        z_dim = code_dim//self.num_split + 128
        self.noise_fc = SpectralNorm(nn.Linear(code_dim//self.num_split, 4 * 4 * in_channels[0]))
    
        self.sa_ids = [int(s.split('B')[-1]) for s in self.blocks_with_attention]
    
        for i, (nc_in, nc_out) in enumerate(zip(in_channels, out_channels)):
            gblock.append(GBlock(nc_in, nc_out, n_class=n_class, z_dim=z_dim, use_attention=(i+1) in self.sa_ids))
        self.blocks = nn.ModuleList(gblock)
    
        self.output_layer_bn = BatchNorm2d(1 * chn, eps=1e-5)
        self.output_layer_conv = SpectralNorm(nn.Conv2d(1 * chn, 3, [3, 3], padding=1))
 
    def get_in_out_channels(self):
        resolution = self.resolution
        if resolution == 1024:
            channel_multipliers = [16, 16, 8, 8, 4, 2, 1, 1, 1]
        elif resolution == 512:
            channel_multipliers = [16, 16, 8, 8, 4, 2, 1, 1]
        elif resolution == 256:
            channel_multipliers = [16, 16, 8, 8, 4, 2, 1]
        elif resolution == 128:
            channel_multipliers = [16, 16, 8, 4, 2, 1]
        elif resolution == 64:
            channel_multipliers = [16, 16, 8, 4, 2]
        elif resolution == 32:
            channel_multipliers = [4, 4, 4, 4]
        else:
            raise ValueError("Unsupported resolution: {}".format(resolution))
        in_channels = [self.chn * c for c in channel_multipliers[:-1]]
        out_channels = [self.chn * c for c in channel_multipliers[1:]]
        return in_channels, out_channels
 
    def forward(self, input, class_id):
        codes = torch.chunk(input, self.num_split, 1)
        class_emb = self.embed_y(class_id)  # 128
        out = self.noise_fc(codes[0])
        out = torch.Tensor.permute(torch.reshape(out,(out.shape[0], 4, 4, -1)),(0, 3, 1, 2))
        for i, (code, gblock) in enumerate(zip(codes[1:], self.blocks)):
            condition = torch.cat([code, class_emb], 1)
            out = gblock(out, condition)
    
        out = self.output_layer_bn(out)
        out = torch.relu(out)
        out = self.output_layer_conv(out)
        
        return (torch.tanh(out) + 1) / 2

    def forward_w(self, ws):
        out = self.noise_fc(ws[0])
        out = torch.Tensor.permute(torch.reshape(out,(out.shape[0], 4, 4, -1)),(0, 3, 1, 2))
        for i, (w, gblock) in enumerate(zip(ws[1:], self.blocks)):
            out = gblock(out, w)
    
        out = self.output_layer_bn(out)
        out = torch.relu(out)
        out = self.output_layer_conv(out)
        
        return (torch.tanh(out) + 1) / 2

    def forward_wp(self, z0, gammas, betas):
        out = self.noise_fc(z0)
        out = torch.Tensor.permute(torch.reshape(out,(out.shape[0], 4, 4, -1)),(0, 3, 1, 2))
        for i, (gamma, beta, gblock) in enumerate(zip(gammas, betas, self.blocks)):
            out = gblock(out, [[gamma[0], beta[0]], [gamma[1], beta[1]]])
    
        out = self.output_layer_bn(out)
        out = torch.relu(out)
        out = self.output_layer_conv(out)
        
        return (torch.tanh(out) + 1) / 2


In [ ]:
generator = Generator(code_dim=140, n_class=1000, chn=96, blocks_with_attention="B5", resolution=256).cuda()

def pt_filter(x):
  if 'weight_v' in x[0] or '._mean' in x[0] or '._variance' in x[0] \
     or'.bn_in_cond.weight' in x[0] or '.bn_in_cond.bias' in x[0]:
    return False
  return True
 
_pt_params = list(filter(pt_filter, generator.named_parameters()))
pt_params = []
for i, params in enumerate(_pt_params):
  b_continue = False
  for j in range(6):
    if 'attention.gamma' in _pt_params[i-j][0]:
      pt_params.append(_pt_params[i+1])
      b_continue = True
  if b_continue:
    continue
  if 'attention.gamma' in _pt_params[i-6][0]:
    pt_params.append(_pt_params[i-6])
    continue
  if 'output_layer.0.weight' in params[0]:
    pt_params.append(_pt_params[i+1])
    continue
  if 'output_layer.0.bias' in params[0]:
    pt_params.append(_pt_params[i-1])
    continue
  pt_params.append(params)
 
_pt_params = pt_params
pt_params = [param for param in _pt_params]
pt_params[-8] = _pt_params[-6]
pt_params[-7] = _pt_params[-5]
pt_params[-6] = _pt_params[-4]
pt_params[-5] = _pt_params[-8]
pt_params[-4] = _pt_params[-7]
 
for i, (tf_weight, pt_param) in enumerate(zip(tf_weights, pt_params)):
  print(pt_param[0], tf_weight[0])
  if len(pt_param[1].shape) == 4:
    weight = tf_weight[1].transpose([3, 2, 0, 1])
  elif len(pt_param[1].shape) == 2 and \
        pt_param[1].shape[0] == tf_weight[1].shape[1] and \
        pt_param[1].shape[1] == tf_weight[1].shape[0]:
    weight = tf_weight[1].transpose([1,0])
  else:
    weight = tf_weight[1].reshape(pt_param[1].shape)
  grad_status = pt_param[1].requires_grad
  if grad_status:
    pt_param[1].requires_grad = False
  pt_param[1].copy_(torch.from_numpy(weight).cuda())
  if grad_status:
    pt_param[1].requires_grad = True
  print(tf_weight[0], tf_weight[1].shape, pt_param[0], pt_param[1].shape)

In [ ]:
save_path = './pytorch_model.bin'
torch.save(generator.state_dict(), save_path)

In [ ]:
from IPython.display import display

generator.eval()
for i in range(32):
  x = torch.randn([1,140]).cuda()
  y = torch.randint(0,1000,size=[1]).cuda().long()
  y_hot = F.one_hot(y, num_classes=1000).float()
  img = np.uint8(generator(x, y_hot).detach().cpu().numpy()[0].transpose(1,2,0).clip(0,1)*255)
  from PIL import Image
  display(Image.fromarray(img))

## Upload Model to HuggingFace Models

In [ ]:
!pip install transformers > /dev/null
!apt-get install git-lfs -y

In [ ]:
!git config --global credential.helper store
!git config --global user.email "HighCWu@163.com"
!git config --global user.name "HighCWu"

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli repo create anime-biggan-pytorch

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/HighCWu/anime-biggan-pytorch
!cp pytorch_model.bin anime-biggan-pytorch/

In [ ]:
!cd anime-biggan-pytorch && git lfs install
!cd anime-biggan-pytorch && git lfs track "*.bin" "*.pkl" "*.pth"

In [ ]:
!cd anime-biggan-pytorch && git add .
!cd anime-biggan-pytorch && git commit -m "First model version"

In [ ]:
!cd anime-biggan-pytorch && git push